# An Introduction to Algorithms in Qiskit

This is an introduction to algorithms in Qiskit and provides a high-level overview to help understand the various aspects of the functionality to get started. Other tutorials will provide more in-depth material, on given algorithms, and ways to use them etc.

## How is the algorithm library structured?

Qiskit provides a number of [Algorithms](https://qiskit.org/documentation/apidoc/algorithms.html) and they are grouped by category according to the task they can perform. For instance `Minimum Eigensolvers` to find the smallest eigen value of an operator, for example ground state energy of a chemistry Hamiltonian or a solution to an optimization problem when expressed as an Ising Hamiltonian. There are `Time Evolers` for the time evolution of quantum systems and `Amplitude Estimators` for value estimation that can be used say in financial applications. The full set of categories can be seen in the Algorithms documentation link above.

Algorithms are configurable and often part of the configuration will be in the form of smaller building blocks, of which different instances of the building block type can be given. For instance with `VQE`, the Variational Quantum Eigensolver, it takes a trial wavefunction, in the form of a `QuantumCircuit` and a classical optimizer among other things.

Let's take a look at an example to construct a VQE instance. Here `TwoLocal` is the variational form (trial wavefunction), a parameterized circuit which can be varied, and `SLSQP` a classical optimizer. These are created as separate instances and passed to VQE when it is constructed. Trying, for example, a different classical optimizer, or variational form is simply a case of creating an instance of the one you want and passing it into VQE.

In [1]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

num_qubits = 2
ansatz = TwoLocal(num_qubits, 'ry', 'cz')
optimizer = SLSQP(maxiter=1000)

Let's draw the ansatz so we can see it's a `QuantumCircuit` where θ\[0\] through θ\[7\] will be the parameters that are varied as VQE optimizer finds the minimum eigenvalue. We'll come back to the parameters later in a working example below.

In [2]:
ansatz.draw()

┌────────────────────────────────────────────────────┐
q_0: ┤0                                                   ├
     │  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
q_1: ┤1                                                   ├
     └────────────────────────────────────────────────────┘

But more is needed before we can run the algorithm so let's get to that next.

## How to run an algorithm?

Algorithms rely on the primitives to evaluate expectation values or sample circuits. The primitives can be based on  a simulator or real device and can be used interchangeably in the algorithms, as they all implement the same interface.

In the VQE, we have to evaluate expectation values, so for example we can use the `qiskit.primitives.Estimator` which is shipped with the default Qiskit Terra installation.

In [3]:
from qiskit.primitives import Estimator

estimator = Estimator()

With all these ingredients ready, we can now instantiate the VQE:

In [4]:
vqe = VQE(estimator, ansatz, optimizer)

Now we would be able to call the [compute_mininum_eigenvalue()](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue) method. The latter is the interface of choice for the application modules, such as Nature and Optimization, in order that they can work interchangeably with any algorithm within the specific category.

## A complete working example

Let's put what we have learned from above together and create a complete working example. VQE will find the minimum eigenvalue, i.e. minimum energy value of a Hamiltonian operator and hence we need such an operator for VQE to work with. Such an operator is given below. This was originally created by the Nature application module as the Hamiltonian for an H2 molecule at 0.735A interatomic distance. It's a sum of Pauli terms as below, but for now I am not going to say anything further about it since the goal is to run the algorithm, but further information on operators can be found in other tutorials.

In [5]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list([
    ("II", -1.052373245772859),
    ("IZ", 0.39793742484318045),
    ("ZI", -0.39793742484318045),
    ("ZZ", -0.01128010425623538),
    ("XX", 0.18093119978423156)
])

So let's run VQE and print the result object it returns.

In [6]:
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 73,
    'eigenvalue': -1.8572749161351831,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x7f9d4177aaa0>,
    'optimal_parameters': {   ParameterVectorElement(θ[1]): 6.517956636024121,
                              ParameterVectorElement(θ[4]): 1.363689589415131,
                              ParameterVectorElement(θ[5]): -2.306189065326123,
                              ParameterVectorElement(θ[2]): -4.006163849091169,
                              ParameterVectorElement(θ[6]): -2.452750165826212,
                              ParameterVectorElement(θ[3]): -0.08386236594758899,
                              ParameterVectorElement(θ[7]): 2.5235737314763385,
                              ParameterVectorElement(θ[0]): -2.980451598834391},
    'optimal_point': array([-2.9804516 ,  6.51795664, -4.00616385, -0.08386237,  1.36368959,
       -2.30618907, -2.45275017,  2.52357373]),
    'optim

From the above result we can see the number of cost function (=energy) evaluations the optimizer took until it found the minimum eigenvalue of $\approx -1.85727$ which is the electronic ground state energy of the given H2 molecule. The optimal parameters of the ansatz can also be seen which are the values that were in the ansatz at the minimum value.

## Updating the primitive inside VQE

To close off let's also change the estimator primitive inside the a VQE. Maybe you're satisfied with the simulation results and now want to use a shot-based simulator, or run on hardware!

In this example we're changing to a shot-based estimator provided by Qiskit Aer. For noisy loss functions, the SPSA optimizer typically performs well, so we also update the optimizer.

In [7]:
from qiskit.algorithms.optimizers import SPSA
from qiskit_aer.primitives import Estimator as AerEstimator

aer_estimator = AerEstimator()

vqe.estimator = aer_estimator
vqe.optimizer = SPSA(maxiter=100)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 200,
    'eigenvalue': -1.8504008230104911,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x7f9d4177ba90>,
    'optimal_parameters': {   ParameterVectorElement(θ[1]): 6.358607077894817,
                              ParameterVectorElement(θ[4]): -0.5063790803882037,
                              ParameterVectorElement(θ[5]): -3.6693961575629945,
                              ParameterVectorElement(θ[2]): 0.6034365009695398,
                              ParameterVectorElement(θ[6]): -3.0008874033354234,
                              ParameterVectorElement(θ[3]): 3.8236241531918003,
                              ParameterVectorElement(θ[7]): 6.108465696139847,
                              ParameterVectorElement(θ[0]): -7.257011246925661},
    'optimal_point': array([-7.25701125,  6.35860708,  0.6034365 ,  3.82362415, -0.50637908,
       -3.66939616, -3.0008874 ,  6.1084657 ]),
    'opt

As the identical seeding was used as the prior example the result can be seen to be the same.

This concludes this introduction to algorithms in Qiskit. Please check out the other algorithm tutorials in this series for both broader as well as more in depth coverage of the algorithms.

In [8]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright